In [85]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [72]:
def prepare_random_state(a, b, c):
    qml.RX(a, wires=0)
    qml.RY(b, wires=0)
    qml.RZ(c, wires=0)

Three-qubit bit flip error correction code with conventional error correction algorithm

<img src="img/Quantum_error_correction_of_bit_flip_using_three_qubits.png" width="800px"> 

In [73]:
def simulate_3_qubit_flip(p, wires):
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[0], wires[2]])

    # simulate bit flip error on the qubits
    qml.BitFlip(p, wires=wires[0])
    qml.BitFlip(p, wires=wires[1])
    qml.BitFlip(p, wires=wires[2])


In [74]:
# Three-qubit bit flip code without QAE for benchmark

dev = qml.device("default.mixed", wires=3)

@qml.qnode(dev)
def ground_truth_3_qubit(a, b, c):
    prepare_random_state(a, b, c)
    return qml.probs(wires=0)


@qml.qnode(dev)
def qec_3_qubit_flip(p, a, b, c):
    prepare_random_state(a, b, c)
    
    simulate_3_qubit_flip(p, [0,1,2])
    # error correction circuit
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.Toffoli(wires=[1,2,0])
    return qml.probs(wires=0)

print(ground_truth_3_qubit(0.1, 0.2, 0.3))
print(qec_3_qubit_flip(0.3, 0.1, 0.2, 0.3))

[0.98758516 0.01241484]
[0.77694837 0.22305163]


three-qubit QAE

<img src="img/3_qubit_qnn.png" width="300px"> 

In [87]:
n_samples= 100
state = np.random.rand(3)
p = 0.1
dev = qml.device("default.mixed", wires=6)
X = np.random.randn(n_samples, 3)

def loss(weights):
    loss_sum = 0.0
    for idx in range(n_samples):
        state = X[idx]
        true_expval = qae_ground_truth(state)
        estimated_expval = three_qubit_qae(state, weights)
        loss_sum += (estimated_expval - true_expval) ** 2
    return loss_sum / n_samples

@qml.qnode(dev)
def qae_ground_truth(state):
    prepare_random_state(state[0], state[1], state[2])
    
    simulate_3_qubit_flip(p, [0,1,2])

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2))

# Three-qubit QAE
@qml.qnode(dev)
def three_qubit_qae(state, weights):
    # encoder reduces the information from three qubits into latent space of one qubit
    # parameters are a, b, and, c which we will tune using gradient descent
    # three qubits
    
    prepare_random_state(state[0], state[1], state[2])
    simulate_3_qubit_flip(p, [0,1,2])


    qml.RX(weights[0], wires=0)
    qml.RY(weights[1], wires=1)
    qml.RZ(weights[2], wires=2)
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[2, 0])

    # decoder uses the encoded information from qubit 0 and decodes this back into three qubits
    qml.RX(weights[3], wires=0)
    qml.RY(weights[4], wires=3)
    qml.RZ(weights[5], wires=4)
    qml.Toffoli(wires=[0, 3, 4])
    qml.Toffoli(wires=[3, 4, 0])
    qml.Toffoli(wires=[4, 0, 3])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(3) @ qml.PauliZ(4))

weights = np.random.rand(6, requires_grad=True)

#print(three_qubit_qae(p, state, weights))
print(loss(weights))

0.006215749939489079


In [88]:
theta = np.array(0., requires_grad=True)
phi = np.array(0., requires_grad=True)

opt = qml.GradientDescentOptimizer()

num_its = 25
running_costs = []

for i in range(num_its):
    weights, _cost = opt.step_and_cost(loss, weights)
#     (theta, phi, zeta), _cost = opt.step_and_cost(circuit, theta, phi, zeta)
    running_costs.append(_cost)
    print(i)

plt.plot(running_costs)

0
1
2
3
4
5
6
7
8
9
10
11


Five qubit code stabilizer measurement circuit


<img src="img/Five_qubit_code_stabilizer_measurement_circuit.png" width="800px"> 